## Environemnt Setup

In [1]:
# Imports 

import pandas as pd 

import numpy as np

## Load the Data

In [2]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.tail()
df.to_csv("raw_population_data.csv")

## Cleaning the data for analysis

In [3]:
# Checking the headers of the dataframe

headers = df.columns.to_list()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [4]:
# Dropping unnecessary columns from the dataframe

drop_headers_list = ['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'C02076V03371',
 'C03789V04537',
 'UNIT',
]
df.drop(columns=drop_headers_list, inplace=True)
df.head()

,Sex,Single Year of Age,Administrative Counties,VALUE
0,Both sexes,All ages,Ireland,5149139
1,Both sexes,All ages,Carlow County Council,61968
2,Both sexes,All ages,Dublin City Council,592713
3,Both sexes,All ages,Dún Laoghaire Rathdown County Council,233860
4,Both sexes,All ages,Fingal County Council,330506


In [5]:
# # Remove 'Both Sexes' from the dataframe for clearer analysis
df = df[df['Sex'] != 'Both sexes']

In [6]:
# Cleaning up the 'Single Age' column

df = df[df["Single Year of Age"] != "All ages"] # Remove the 'All ages' rows
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0') # Replace 'Under 1 year' with '0'
df['Single Year of Age'] = df['Single Year of Age'].str.replace(r'\D', '', regex=True) # Remove any non-digit characters using regex
# df['Single Year of Age'].unique() # Check unique values after cleaning

In [7]:
# Converting 'Single Year of Age' to integer type

# df.info() # checking the data types and non-null counts
df['Single Year of Age'] = df['Single Year of Age'].astype('Int64') # Convert 'Single Year of Age' to integer
df.info() # checking the data types and non-null counts after conversion

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 3296 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      6464 non-null   object
 1   Single Year of Age       6464 non-null   Int64 
 2   Administrative Counties  6464 non-null   object
 3   VALUE                    6464 non-null   int64 
dtypes: Int64(1), int64(1), object(2)
memory usage: 258.8+ KB


### Checking there are no conflicts or duplicating of Data 

In [8]:
admin_districts = df['Administrative Counties'].unique().tolist()
admin_districts

['Ireland',
 'Carlow County Council',
 'Dublin City Council',
 'Dún Laoghaire Rathdown County Council',
 'Fingal County Council',
 'South Dublin County Council',
 'Kildare County Council',
 'Kilkenny County Council',
 'Laois County Council',
 'Longford County Council',
 'Louth County Council',
 'Meath County Council',
 'Offaly County Council',
 'Westmeath County Council',
 'Wexford County Council',
 'Wicklow County Council',
 'Clare County Council',
 'Cork City Council',
 'Cork County Council',
 'Kerry County Council',
 'Limerick City & County Council',
 'Tipperary County Council',
 'Waterford City & County Council',
 'Galway City Council',
 'Galway County Council',
 'Leitrim County Council',
 'Mayo  County Council',
 'Roscommon County Council',
 'Sligo County Council',
 'Cavan County Council',
 'Donegal County Council',
 'Monaghan County Council']

In [9]:
# look at the ireland district data
ireland_data = df[df['Administrative Counties'] == 'Ireland']
ireland_data

,Sex,Single Year of Age,Administrative Counties,VALUE
3296,Male,0,Ireland,29610
3328,Male,1,Ireland,28875
3360,Male,2,Ireland,30236
3392,Male,3,Ireland,31001
3424,Male,4,Ireland,31686
...,...,...,...,...
9632,Female,96,Ireland,956
9664,Female,97,Ireland,732
9696,Female,98,Ireland,492
9728,Female,99,Ireland,336


In [10]:
# checking if the Irleand data matches the total of all admin districts
ireland_total = ireland_data['VALUE'].sum()
regions_total = df.loc[df['Administrative Counties'] != 'Ireland', 'VALUE'].sum()
ireland_total == regions_total # should return True if they match 


np.True_

As the Ireland Total is equal to all others we should remove it from the df before running the analysis using the regions total. Note coud just take this to calcualted the weight averages for the Sex but then the regions couldn't be analysised. 

## Part 1

Brief: 
> Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.  
> Weighted mean age (by sex)  
> The difference between the sexes by age  
>This part does not need to look at the regions.  

In [11]:
# Creating a Pivot Table to analyze population by Age and Sex 

# Removing 'Ireland' from the dataframe for analysis
df = df[df['Administrative Counties'] != 'Ireland']

# added .astype(int) to convert float to int as I was getting float values 
sexes_age_pivot = pd.pivot_table(df, 
                            values='VALUE', 
                            index="Single Year of Age", 
                            columns= "Sex",
                            ).astype(int) 
# df_analysis.info()
sexes_age_pivot.to_csv("population_by_age_and_sex.csv")

In [12]:
sexes_age_pivot.head()

Sex,Female,Male
Single Year of Age,,
0,909,955
1,888,931
2,934,975
3,951,1000
4,961,1022


In [13]:
# calculating the weighted mean for the ages of the population (ignoring the administrative district)

weighted_mean_age = np.average(sexes_ages.index, weights=

_IncompleteInputError: incomplete input (2679707194.py, line 3)

# for other parts

In [ ]:
regional_analysis = pd.pivot_table(df,
                                 values='VALUE',
                                 index=['Administrative Counties', 'Single Year of Age'],
                                 columns='Sex',
                                 aggfunc='sum').astype(int)

# Calculate difference between sexes for each region and age
regional_analysis['Gender_Difference'] = regional_analysis['Female'] - regional_analysis['Male']


In [ ]:
# Get the largest differences by region
largest_differences = regional_analysis.groupby('Administrative Counties')['Gender_Difference'].agg(['max', 'min'])
largest_differences.columns = ['Largest Female Surplus', 'Largest Male Surplus']
print("Largest gender differences by region:")
print(largest_differences)

In [ ]:

# Find which ages these differences occur at
def get_max_diff_age(group):
    max_idx = group['Gender_Difference'].idxmax()
    min_idx = group['Gender_Difference'].idxmin()
    return pd.Series({
        'Age (Max Female Surplus)': max_idx[1],
        'Age (Max Male Surplus)': min_idx[1]
    })

age_at_max_diff = regional_analysis.groupby('Administrative Counties').apply(get_max_diff_age)
print("\nAges at which maximum differences occur:")
print(age_at_max_diff)